In [ ]:
import time
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import mlrose

import warnings
warnings.filterwarnings("ignore")

In [4]:
problems = ['OneMax','FourPeaks','Knapsack']
algs = ['SA','RHC','MIMIC','GA']
sa_cooling = ['GeomDecay', 'ArithDecay','ExpDecay']
ga_population = [50, 100, 200, 400]
ga_mutation_prob = [0.0001, 0.01, 0.1, 0.2, 0.5, 0.9]
mimic_population = ga_population
mimic_keep_pct = [0.30,0.50,0.70,0.90]
sizes = [r for r in range(10,65,5)]
experiments = 5

results = []

In [5]:
for n in sizes:
    for p in problems:
        if p == 'Knapsack':
            fitness = mlrose.Knapsack(
                weights=np.random.uniform(size = n), 
                values=np.arange(1, n+1, 1))
        elif p == 'FourPeaks':
            fitness = mlrose.FourPeaks()
        elif p == 'OneMax':
            fitness = mlrose.OneMax()
        
        problem_fit = mlrose.DiscreteOpt(
            length = n,
            fitness_fn = fitness,
            maximize = True,
            max_val = 2 # binary
        )
        
        for a in algs:
            if a == 'RHC':        
                times, optima, iterations = [], [], []
                for e in range(experiments):
                    start = time.time()
                    best_state, best_fitness, curve = mlrose.random_hill_climb(
                        problem_fit,
                        curve=True)
                    times.append(time.time() - start)
                    optima.append(best_fitness)
                    iterations.append(curve.shape[0])
                        
                results.append({'algorithm': a,
                                'problem': p,
                                'mean_time': np.mean(times),
                                'mean_optimum': np.mean(optima),
                                'mean_iterations': np.mean(iterations),
                                'problem_size': n
                                })
            if a == 'SA':
                for opt in sa_cooling:
                    decay = getattr(mlrose, opt)()
                    times, optima, iterations = [], [], []
                    for e in range(experiments):
                        start = time.time()
                        
                        best_state, best_fitness, curve = mlrose.simulated_annealing(
                            problem_fit,
                            schedule=decay,
                            curve=True)
                        times.append(time.time() - start)
                        optima.append(best_fitness)
                        iterations.append(curve.shape[0])
                        
                    results.append({'algorithm': a,
                                    'problem': p,
                                    'mean_time': np.mean(times),
                                    'mean_optimum': np.mean(optima),
                                    'mean_iterations': np.mean(iterations),
                                    'problem_size': n,
                                    'cooling_schedule': opt,
                                   })

            if a == 'GA':
                for pop in ga_population:
                    for prob in ga_mutation_prob:
                        decay = getattr(mlrose, opt)()
                        times, optima, iterations = [], [], []
                        for e in range(experiments):
                            start = time.time()
                        
                            best_state, best_fitness, curve = mlrose.genetic_alg(
                                problem_fit,
                                pop_size = pop,
                                mutation_prob = prob,
                                curve = True
                                )
                            times.append(time.time() - start)
                            optima.append(best_fitness)
                            iterations.append(curve.shape[0])
                        
                        results.append({'algorithm': a,
                                        'problem': p,
                                        'mean_time': np.mean(times),
                                        'mean_optimum': np.mean(optima),
                                        'mean_iterations': np.mean(iterations),
                                        'problem_size': n,
                                        'population': pop,
                                        'mutation_prob': prob
                                       })

            if a == 'MIMIC':
                for pop in mimic_population:
                    for keep in mimic_keep_pct:
                        decay = getattr(mlrose, opt)()
                        times, optima, iterations = [], [], []
                        for e in range(experiments):
                            start = time.time()
                            best_state, best_fitness, curve = mlrose.mimic(
                                problem_fit,
                                pop_size = pop,
                                keep_pct = keep,
                                fast_mimic = True,
                                curve = True
                            )
                            times.append(time.time() - start)
                            optima.append(best_fitness)
                            iterations.append(curve.shape[0])
                        
                        results.append({'algorithm': a,
                                        'problem': p,
                                        'mean_time': np.mean(times),
                                        'mean_optimum': np.mean(optima),
                                        'mean_iterations': np.mean(iterations),
                                        'problem_size': n,
                                        'population': pop,
                                        'samples_kept': keep
                                       })
        


In [6]:
results

[{'algorithm': 'SA',
  'problem': 'OneMax',
  'mean_time': 0.004886341094970703,
  'mean_optimum': 9.0,
  'mean_iterations': 106.8,
  'problem_size': 10,
  'cooling_schedule': 'GeomDecay'},
 {'algorithm': 'SA',
  'problem': 'OneMax',
  'mean_time': 0.038310670852661134,
  'mean_optimum': 8.2,
  'mean_iterations': 1276.4,
  'problem_size': 10,
  'cooling_schedule': 'ArithDecay'},
 {'algorithm': 'SA',
  'problem': 'OneMax',
  'mean_time': 0.004890060424804688,
  'mean_optimum': 9.6,
  'mean_iterations': 168.8,
  'problem_size': 10,
  'cooling_schedule': 'ExpDecay'},
 {'algorithm': 'RHC',
  'problem': 'OneMax',
  'mean_time': 0.0004901885986328125,
  'mean_optimum': 9.0,
  'mean_iterations': 23.8,
  'problem_size': 10},
 {'algorithm': 'MIMIC',
  'problem': 'OneMax',
  'mean_time': 0.033520984649658206,
  'mean_optimum': 9.2,
  'mean_iterations': 11.2,
  'problem_size': 10,
  'population': 50,
  'samples_kept': 0.3},
 {'algorithm': 'MIMIC',
  'problem': 'OneMax',
  'mean_time': 0.037746143

In [56]:
data = {}
for p in problems:
    for c in sa_cooling:
        metrics = []
        for s in sizes:
            l = [i['mean_optimum'] for i in results if i["problem_size"] == s and
                 i['problem'] == p and
                 i['algorithm'] == 'SA' and
                 i['cooling_schedule'] == c]
            metrics.append(l[0])
        data[c] = metrics

    figsize = (15,4)
    fig, ax = plt.subplots(figsize=figsize)
    pd.DataFrame(data, index=sizes).plot(marker='o', ax=ax)
    ax.set_xlabel("Problem size")
    ax.set_ylabel("Mean fitness")
    ax.set_title("Comparing SA cooling decay for {}".format(p))
    fig.savefig("figures/SA_{}_{}_comp.png".format(p, c), bbox_inches='tight')

In [57]:
data = {}
for p in problems:
    for r in ga_population:
        for m in ga_mutation_prob:
            metrics = []
            for s in sizes:
                l = [i['mean_optimum'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == 'GA' and
                     i['population'] == r and
                     i['mutation_prob'] == m]
                metrics.append(l[0])
            data[m] = metrics

        figsize = (15,4)
        fig, ax = plt.subplots(figsize=figsize)
        pd.DataFrame(data, index=sizes).plot(marker='o', ax=ax)
        ax.set_xlabel("Problem size")
        ax.set_ylabel("Mean fitness")
        ax.set_title("Comparing GA at population {} for {}".format(r, p))
        fig.savefig("figures/GA_{}_pop{}_comp.png".format(p, r),bbox_inches='tight')

In [58]:
data = {}
for p in problems:
    for r in mimic_population:
        for m in mimic_keep_pct:
            metrics = []
            for s in sizes:
                l = [i['mean_optimum'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == 'MIMIC' and
                     i['population'] == r and
                     i['samples_kept'] == m]
                metrics.append(l[0])
            data[m] = metrics

        figsize = (15,4)
        fig, ax = plt.subplots(figsize=figsize)
        pd.DataFrame(data, index=sizes).plot(marker='o', ax=ax)
        ax.set_xlabel("Problem size")
        ax.set_ylabel("Mean fitness")
        ax.set_title("Comparing MIMIC at population {} for {}".format(r, p))
        fig.savefig("figures/MIMIC_{}_pop{}_comp.png".format(p, r),bbox_inches='tight')

In [59]:
data = {}
for p in problems:
    metrics = []
    for s in sizes:
        l = [i['mean_optimum'] for i in results if i["problem_size"] == s and
             i['problem'] == p and
             i['algorithm'] == 'RHC']
        metrics.append(l[0])
    data[p] = metrics

figsize = (15,4)
fig, ax = plt.subplots(figsize=figsize)
pd.DataFrame(data, index=sizes).plot(marker='o', ax=ax)
ax.set_xlabel("Problem size")
ax.set_ylabel("Mean fitness")
ax.set_title("Comparing RHC across all problems")
fig.savefig("figures/RHC_comp.png",bbox_inches='tight')

In [60]:
#comparing optima for all

cooling = 'GeomDecay'
pop = 200
mut = 0.01
kept = 0.3

data = {}
for p in problems:
    for a in algs:
        metrics = []
        for s in sizes:
            if a == 'RHC':
                l = [i['mean_optimum'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == a]
            elif a == 'SA':
                l = [i['mean_optimum'] for i in results if i["problem_size"] == s and
                 i['problem'] == p and
                 i['algorithm'] == a and
                 i['cooling_schedule'] == cooling]
            elif a == 'GA':
                l = [i['mean_optimum'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == 'GA' and
                     i['population'] == pop and
                     i['mutation_prob'] == mut]
            elif a == 'MIMIC':
                l = [i['mean_optimum'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == 'MIMIC' and
                     i['population'] == pop and
                     i['samples_kept'] == kept]
                
            metrics.append(l[0])
        data[p] = metrics

        figsize = (15,4)
        fig, ax = plt.subplots(figsize=figsize)
        pd.DataFrame(data, index=sizes).plot(marker='o', ax=ax)
        ax.set_xlabel("Problem size")
        ax.set_ylabel("Mean fitness")
        ax.set_title("Comparing the optima for {} for all problems".format(a))
        fig.savefig("figures/all_{}_optima.png".format(a),bbox_inches='tight')

In [78]:
#comparing times for all
cooling = 'GeomDecay'
pop = 200
mut = 0.01
kept = 0.3

data = {}
for p in problems:
    for a in algs:
        metrics = []
        for s in sizes:
            if a == 'RHC':
                l = [i['mean_time'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == a]
            elif a == 'SA':
                l = [i['mean_time'] for i in results if i["problem_size"] == s and
                 i['problem'] == p and
                 i['algorithm'] == a and
                 i['cooling_schedule'] == cooling]
            elif a == 'GA':
                l = [i['mean_time'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == 'GA' and
                     i['population'] == pop and
                     i['mutation_prob'] == mut]
            elif a == 'MIMIC':
                l = [i['mean_time'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == 'MIMIC' and
                     i['population'] == pop and
                     i['samples_kept'] == kept]
                
            metrics.append(l[0])
        data[p] = metrics

        figsize = (15,4)
        fig, ax = plt.subplots(figsize=figsize)
        pd.DataFrame(data, index=sizes).plot(marker='o', ax=ax)
        ax.set_xlabel("Problem size")
        ax.set_ylabel("Mean execution time")
        ax.set_title("Comparing the times for {} for all problems".format(a))
        fig.savefig("figures/all_{}_time.png".format(a),bbox_inches='tight')

In [79]:
#comparing iterations for all
cooling = 'GeomDecay'
pop = 200
mut = 0.01
kept = 0.3

data = {}
for p in problems:
    for a in algs:
        metrics = []
        for s in sizes:
            if a == 'RHC':
                l = [i['mean_iterations'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == a]
            elif a == 'SA':
                l = [i['mean_iterations'] for i in results if i["problem_size"] == s and
                 i['problem'] == p and
                 i['algorithm'] == a and
                 i['cooling_schedule'] == cooling]
            elif a == 'GA':
                l = [i['mean_iterations'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == 'GA' and
                     i['population'] == pop and
                     i['mutation_prob'] == mut]
            elif a == 'MIMIC':
                l = [i['mean_iterations'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == 'MIMIC' and
                     i['population'] == pop and
                     i['samples_kept'] == kept]
                
            metrics.append(l[0])
        data[p] = metrics

        figsize = (15,4)
        fig, ax = plt.subplots(figsize=figsize)
        pd.DataFrame(data, index=sizes).plot(marker='o', ax=ax)
        ax.set_xlabel("Problem size")
        ax.set_ylabel("Mean iterations")
        ax.set_title("Comparing the iterations for {} for all problems".format(a))
        fig.savefig("figures/all_{}_iterations.png".format(a),bbox_inches='tight')

In [80]:
#all optima minus knapsack

cooling = 'GeomDecay'
pop = 200
mut = 0.01
kept = 0.3

data = {}
for p in ['OneMax', 'FourPeaks']:
    for a in algs:
        metrics = []
        for s in sizes:
            if a == 'RHC':
                l = [i['mean_optimum'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == a]
            elif a == 'SA':
                l = [i['mean_optimum'] for i in results if i["problem_size"] == s and
                 i['problem'] == p and
                 i['algorithm'] == a and
                 i['cooling_schedule'] == cooling]
            elif a == 'GA':
                l = [i['mean_optimum'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == 'GA' and
                     i['population'] == pop and
                     i['mutation_prob'] == mut]
            elif a == 'MIMIC':
                l = [i['mean_optimum'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == 'MIMIC' and
                     i['population'] == pop and
                     i['samples_kept'] == kept]
                
            metrics.append(l[0])
        data[p] = metrics

        figsize = (15,4)
        fig, ax = plt.subplots(figsize=figsize)
        pd.DataFrame(data, index=sizes).plot(marker='o', ax=ax)
        ax.set_xlabel("Problem size")
        ax.set_ylabel("Mean fitness")
        ax.set_title("Comparing the optima for {} for just OneMax and FourPeaks".format(a))
        fig.savefig("figures/all_nokn_{}_optima.png".format(a),bbox_inches='tight')

In [81]:
#comparing iterations for all algs
cooling = 'GeomDecay'
pop = 200
mut = 0.01
kept = 0.3

data = {}
for a in algs:
    for p in problems:
        metrics = []
        for s in sizes:
            if a == 'RHC':
                l = [i['mean_iterations'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == a]
            elif a == 'SA':
                l = [i['mean_iterations'] for i in results if i["problem_size"] == s and
                 i['problem'] == p and
                 i['algorithm'] == a and
                 i['cooling_schedule'] == cooling]
            elif a == 'GA':
                l = [i['mean_iterations'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == 'GA' and
                     i['population'] == pop and
                     i['mutation_prob'] == mut]
            elif a == 'MIMIC':
                l = [i['mean_iterations'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == 'MIMIC' and
                     i['population'] == pop and
                     i['samples_kept'] == kept]
                
            metrics.append(l[0])
        data[a] = metrics

        figsize = (15,4)
        fig, ax = plt.subplots(figsize=figsize)
        pd.DataFrame(data, index=sizes).plot(marker='o', ax=ax)
        ax.set_xlabel("Problem size")
        ax.set_ylabel("Mean iterations")
        ax.set_title("Comparing the iterations for {} for all problems".format(p))
        fig.savefig("figures/all_{}_iterations.png".format(p),bbox_inches='tight')

In [84]:
#comparing times for all algs
cooling = 'GeomDecay'
pop = 200
mut = 0.01
kept = 0.3

data = {}
for a in algs:
    for p in problems:
        metrics = []
        for s in sizes:
            if a == 'RHC':
                l = [i['mean_time'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == a]
            elif a == 'SA':
                l = [i['mean_time'] for i in results if i["problem_size"] == s and
                 i['problem'] == p and
                 i['algorithm'] == a and
                 i['cooling_schedule'] == cooling]
            elif a == 'GA':
                l = [i['mean_time'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == 'GA' and
                     i['population'] == pop and
                     i['mutation_prob'] == mut]
            elif a == 'MIMIC':
                l = [i['mean_time'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == 'MIMIC' and
                     i['population'] == pop and
                     i['samples_kept'] == kept]
                
            metrics.append(l[0])
        data[a] = metrics

        figsize = (15,4)
        fig, ax = plt.subplots(figsize=figsize)
        pd.DataFrame(data, index=sizes).plot(marker='o', ax=ax)
        ax.set_xlabel("Problem size")
        ax.set_ylabel("Mean execution time")
        ax.set_title("Comparing the times for {} for all problems".format(p))
        fig.savefig("figures/all_{}_times.png".format(p),bbox_inches='tight')

In [85]:
#comparing optima for all algs
cooling = 'GeomDecay'
pop = 200
mut = 0.01
kept = 0.3

data = {}
for a in algs:
    for p in problems:
        metrics = []
        for s in sizes:
            if a == 'RHC':
                l = [i['mean_optimum'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == a]
            elif a == 'SA':
                l = [i['mean_optimum'] for i in results if i["problem_size"] == s and
                 i['problem'] == p and
                 i['algorithm'] == a and
                 i['cooling_schedule'] == cooling]
            elif a == 'GA':
                l = [i['mean_optimum'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == 'GA' and
                     i['population'] == pop and
                     i['mutation_prob'] == mut]
            elif a == 'MIMIC':
                l = [i['mean_optimum'] for i in results if i["problem_size"] == s and
                     i['problem'] == p and
                     i['algorithm'] == 'MIMIC' and
                     i['population'] == pop and
                     i['samples_kept'] == kept]
                
            metrics.append(l[0])
        data[a] = metrics

        figsize = (15,4)
        fig, ax = plt.subplots(figsize=figsize)
        pd.DataFrame(data, index=sizes).plot(marker='o', ax=ax)
        ax.set_xlabel("Problem size")
        ax.set_ylabel("Mean fitness")
        ax.set_title("Comparing the fitness for {} for all problems".format(p))
        fig.savefig("figures/all_{}_optima.png".format(p),bbox_inches='tight')